#Problem Statement



*   Buat model machine learning untuk mengklasifikasikan sentiment dari sample_data.csv pada link akses data yang diberikan.

*   Lakukan segala macam preprocessing dan analisis yang dibutuhkan untuk modeling data.

*   Tunjukan nilai akurasi dan confusion matrix dari model yang dibentuk.






#About Data Set


*   Data adalah data sentiment yang diberikan oleh Celerates
*   Deskripsi Data :

1.   Sentiment : label of dataset (Positive or Negative)
2.   Customer Review : Raw data of customer review
3. corpus : Lowercase version of Customer Review

#Importing Libraries

In [43]:
import pandas as pd
import string, re

from tqdm import tqdm
tqdm.pandas()

!pip install Sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory

from time import time
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_extraction.text import TfidfVectorizer

from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
#load csv file using pandas
df = pd.read_csv("/content/drive/MyDrive/Celerates/tugas_2.csv")

In [23]:
df.head()

,Sentiment,Customer Review,corpus
0,Positive,"Barang bagus, pengiriman juga cepat. Cuma minu...","barang bagus, pengiriman juga cepat. cuma minu..."
1,Positive,LENGKAP BAGUS PENGIRIMAN CEPAT DAN TEPAT TERPE...,lengkap bagus pengiriman cepat dan tepat terpe...
2,Positive,"Bahan tipis, seller ramah, ekspedisi cpt","bahan tipis, seller ramah, ekspedisi cpt"
3,Positive,produk asli original. suplemen bagus tanpa efe...,produk asli original. suplemen bagus tanpa efe...
4,Positive,Model dan warna aku suka tp pemasangan sedikit...,model dan warna aku suka tp pemasangan sedikit...


In [24]:
df.shape

(1200, 3)

In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Sentiment        1200 non-null   object
 1   Customer Review  1200 non-null   object
 2   corpus           1200 non-null   object
dtypes: object(3)
memory usage: 28.2+ KB


In [32]:
print(df.Sentiment.value_counts())

Sentiment
Positive    600
Negative    600
Name: count, dtype: int64


Sudah menjadi sentiment, sudah ada casefolding (corpus), sudah downsampling.

#Data Preprocessing

##Punctuation

In [33]:
df['corpus'] = df.corpus.str.translate(str.maketrans('', '', string.punctuation))
print(df.shape)
df.head()

(1200, 3)


,Sentiment,Customer Review,corpus
0,Positive,"Barang bagus, pengiriman juga cepat. Cuma minu...",barang bagus pengiriman juga cepat cuma minus ...
1,Positive,LENGKAP BAGUS PENGIRIMAN CEPAT DAN TEPAT TERPE...,lengkap bagus pengiriman cepat dan tepat terpe...
2,Positive,"Bahan tipis, seller ramah, ekspedisi cpt",bahan tipis seller ramah ekspedisi cpt
3,Positive,produk asli original. suplemen bagus tanpa efe...,produk asli original suplemen bagus tanpa efek...
4,Positive,Model dan warna aku suka tp pemasangan sedikit...,model dan warna aku suka tp pemasangan sedikit...


In [36]:
pth = '/content/drive/MyDrive/Celerates/'

In [37]:
slang_dict = pd.read_csv(pth+'kamusalay.csv', encoding='latin1',names=['alay','normal'])
slang_dict = dict(zip(slang_dict.alay.tolist(),slang_dict.normal.tolist()))

In [38]:
def cvt_slang(sentence):
  return ''.join(' ').join(t for t in [slang_dict[x] if x in slang_dict.keys() else x for x in sentence.split()])

In [39]:
df['corpus'] = df.corpus.apply(cvt_slang)
print(df.shape)
df.head()

(1200, 3)


,Sentiment,Customer Review,corpus
0,Positive,"Barang bagus, pengiriman juga cepat. Cuma minu...",barang bagus pengiriman juga cepat cuma minus ...
1,Positive,LENGKAP BAGUS PENGIRIMAN CEPAT DAN TEPAT TERPE...,lengkap bagus pengiriman cepat dan tepat terpe...
2,Positive,"Bahan tipis, seller ramah, ekspedisi cpt",bahan tipis seller ramah ekspedisi cepat
3,Positive,produk asli original. suplemen bagus tanpa efe...,produk asli original suplemen bagus tanpa efek...
4,Positive,Model dan warna aku suka tp pemasangan sedikit...,model dan warna aku suka tetapi pemasangan sed...


##Repetition Character

In [40]:
def repetition_char(txt):
  sub_str = re.findall(r"((\w)\2{2,})",txt)
  for x in sub_str:
    txt = txt.replace(x[0],x[1])
  return txt

In [41]:
df['corpus'] = df.corpus.apply(repetition_char)
df = df.reset_index(drop=True)
print(df.shape)
df.head()

(1200, 3)


,Sentiment,Customer Review,corpus
0,Positive,"Barang bagus, pengiriman juga cepat. Cuma minu...",barang bagus pengiriman juga cepat cuma minus ...
1,Positive,LENGKAP BAGUS PENGIRIMAN CEPAT DAN TEPAT TERPE...,lengkap bagus pengiriman cepat dan tepat terpe...
2,Positive,"Bahan tipis, seller ramah, ekspedisi cpt",bahan tipis seller ramah ekspedisi cepat
3,Positive,produk asli original. suplemen bagus tanpa efe...,produk asli original suplemen bagus tanpa efek...
4,Positive,Model dan warna aku suka tp pemasangan sedikit...,model dan warna aku suka tetapi pemasangan sed...


##Stemming

In [44]:
factory = StemmerFactory()
stemmer = factory.create_stemmer()

df['corpus'] = df.corpus.progress_apply(lambda x : stemmer.stem(str(x)))
print(df.shape)
df.head()

100%|██████████| 1200/1200 [03:45<00:00,  5.32it/s]

(1200, 3)


,Sentiment,Customer Review,corpus
0,Positive,"Barang bagus, pengiriman juga cepat. Cuma minu...",barang bagus kirim juga cepat cuma minus di pa...
1,Positive,LENGKAP BAGUS PENGIRIMAN CEPAT DAN TEPAT TERPE...,lengkap bagus kirim cepat dan tepat terpercaya...
2,Positive,"Bahan tipis, seller ramah, ekspedisi cpt",bahan tipis seller ramah ekspedisi cepat
3,Positive,produk asli original. suplemen bagus tanpa efe...,produk asli original suplemen bagus tanpa efek...
4,Positive,Model dan warna aku suka tp pemasangan sedikit...,model dan warna aku suka tetapi pasang sedikit...


##Stopword

In [45]:
stops = pd.read_csv(pth+'stopwordbahasa.csv',header=None)
stop_factory = StopWordRemoverFactory()
STOPS = stop_factory.get_stop_words()+stops[0].tolist()
STOPS = set(STOPS)

In [46]:
df['corpus'] = df.corpus.progress_apply(lambda c : ''.join(' ').join(x for x in c.split() if x not in STOPS))
print(df.shape)
df.head()

100%|██████████| 1200/1200 [00:00<00:00, 129898.18it/s]

(1200, 3)


,Sentiment,Customer Review,corpus
0,Positive,"Barang bagus, pengiriman juga cepat. Cuma minu...",barang bagus kirim cepat minus packaging nya r...
1,Positive,LENGKAP BAGUS PENGIRIMAN CEPAT DAN TEPAT TERPE...,lengkap bagus kirim cepat terpercayaa nomor 1 ...
2,Positive,"Bahan tipis, seller ramah, ekspedisi cpt",bahan tipis seller ramah ekspedisi cepat
3,Positive,produk asli original. suplemen bagus tanpa efe...,produk asli original suplemen bagus efek sampi...
4,Positive,Model dan warna aku suka tp pemasangan sedikit...,model warna suka pasang ribet moga awet


In [47]:
#save to csv
df.to_csv(pth+'tugas_2_bersih.csv',index=False)

#Sentiment Classification

In [48]:
df = pd.read_csv(pth+'data_bersih.csv')
print(df.shape)
df.head()

(1200, 3)


,reviewContent,sentiment,corpus
0,mantap.. semoga awet barangnya.. maaf baru kas...,positif,mantap moga awet barang maaf kasih nilai
1,Seneng baranga cepat sampai packing juga rapih...,positif,senang baranga cepat packing rapi terima kasih
2,"Barang sampai dengan selamat,. berfungsi denga...",positif,barang selamat fungsi
3,Brg bagus cukup memuaskan,positif,barang bagus muas
4,Sesuai dengan pesanan,positif,sesuai pesan


In [49]:
df.isnull().sum()

reviewContent    0
sentiment        0
corpus           6
dtype: int64

In [50]:
df = df.dropna()
print(df.shape)
print(df.sentiment.value_counts())

(1194, 3)
sentiment
positif    597
negatif    597
Name: count, dtype: int64


In [51]:
x, y =df.corpus, df.sentiment

In [52]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(x)

#Modelling

In [53]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.3)
print("train size:",x_train.shape)
print("test size:",x_test.shape)

train size: (835, 2181)
test size: (359, 2181)


In [54]:
from sklearn.svm import SVC

In [55]:
st = time()
model = SVC()
model.fit(x_train,y_train)
print("[DONE] training process finished:",time()-st,"second(s)\n")
y_pred = model.predict(x_test)
print(classification_report(y_test,y_pred))
print(confusion_matrix(y_test,y_pred))

[DONE] training process finished: 0.14606952667236328 second(s)

              precision    recall  f1-score   support

     negatif       0.83      0.92      0.88       172
     positif       0.92      0.83      0.87       187

    accuracy                           0.87       359
   macro avg       0.88      0.88      0.87       359
weighted avg       0.88      0.87      0.87       359

[[159  13]
 [ 32 155]]


In [58]:
def save_model(model, prefix=''):
    import joblib
    from datetime import datetime

    # Get the current date and time as a string to define the file name
    current_datetime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    filename = f"{prefix}model_{current_datetime}.joblib"

    joblib.dump(model, filename)

    print(f"Model saved to {filename}")

In [59]:
save_model(model, prefix='svm_')

Model saved to svm_model_2024-05-03_16-25-47.joblib
